In [1]:
cd E:/GitHub/computervision

E:\GitHub\computervision


In [2]:
import torch
import os
# os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

from torch.nn import CrossEntropyLoss
from datetime import datetime
from torch.optim import lr_scheduler, SGD, Adamax
from modules.training_util import PytorchDataset, PytorchTraining, EarlyStopping
from modules.models_util import ViT

In [3]:
# Variables for continuing previous training
CONTINUE_TRAINING_DIR = None
# CONTINUE_TRAINING_DIR = 'notebooks/vit/models/'
MODEL_WEIGHTS_PATH = None
if CONTINUE_TRAINING_DIR:
    MODEL_WEIGHTS_PATH = os.path.join(CONTINUE_TRAINING_DIR,'epoch_15.pth')

# CONFIG VARIABLES
FULLY_TRAIN = True
DATA_DIR = [
    'E:/Datasets/cor-splits/sgkf-8-1-1-size-540',
    'E:/Datasets/cor-splits/balanced_train',
    'E:/Datasets/cor-splits/imbalanced_train'
    ]
START_EPOCH = 1 # Change it from 1 if you're continuing an existing run
N_EPOCHS = 2
BATCH_SIZE = 32
SAVE_INTERVAL = 2 # How many epochs between each backup
## Learning Rate Scheduler
LR = 0.001  # When continuing a existing run, update it to the last epoch LR value
LR_EPOCH_SCHEDULE = 10  # Number of epochs before altering LR
LR_GAMMA = 0.1
## Early Stopping
PATIENCE = 10
MIN_DELTA = 0
MIN_EPOCH = 10 # Number of spochs before starting patience counter

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
archs = ['b32']

In [5]:
for dir in DATA_DIR:
    for arch in archs:
        vit = ViT(architecture=arch)
        
        model = vit.load(MODEL_WEIGHTS_PATH, FULLY_TRAIN)
        model = model.to(device)
        
        criterion = CrossEntropyLoss()

        if FULLY_TRAIN:
            params = model.parameters()
        else:
            params = model.heads.parameters()
            
        # optimizer = optim.SGD(model.classifier.parameters(), lr=LR)
        optimizer = SGD(params, lr=LR)
        # optimizer = Adamax(model.parameters(), lr=LR)

        step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=LR_EPOCH_SCHEDULE, gamma=LR_GAMMA)
        es = EarlyStopping(patience=PATIENCE, delta=MIN_DELTA, min_epoch=MIN_EPOCH)
        dataset = PytorchDataset(dir, vit.data_transforms, BATCH_SIZE)

        working_folder = 'notebooks/vit'
        models_folder = 'models'

        curr_time = datetime.now()
        curr_time = curr_time.strftime("%Y-%m-%d %H-%M-%S")

        output_folder = os.path.join(working_folder, models_folder)
        output_folder = os.path.join(output_folder, curr_time)
        
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)

        trainer = PytorchTraining(device, dataset, output_folder)
        model = trainer.train_pytorch_model(model, criterion, optimizer, step_lr_scheduler, es, START_EPOCH, N_EPOCHS, SAVE_INTERVAL)

All parameters for model VisionTransformer requires grad.
Saving model to folder notebooks/vit\models\2024-02-17 22-00-08

Model VisionTransformer
Fully Trained = True
Dataset E:/Datasets/cor-splits/sgkf-8-1-1-size-540
Learning Rate Epoch Schedule = 10
Learning Rate Gamma = 0.1
Optimizer = SGD
Batch Size = 32


Epoch 1/2
----------



c:\Users\Adm\anaconda3\envs\computervision\Lib\site-packages\torch\nn\functional.py:5476: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)


train Loss: 0.7055 Acc: 0.5509
val Loss: 0.5717 Acc: 0.6852
Epoch duration: 0 m 11s
Learning Rate = 0.001



Epoch 2/2
----------

train Loss: 0.5918 Acc: 0.7106
val Loss: 0.5007 Acc: 0.8704
Epoch duration: 0 m 9s
Learning Rate = 0.001


Training complete in 0m 20s
Best val Acc: 0.870370

Confusion Matrix
[[38 16]
 [ 8 46]]

Accuracy by class
[70.37037037 85.18518519]
All parameters for model VisionTransformer requires grad.
Saving model to folder notebooks/vit\models\2024-02-17 22-00-34

Model VisionTransformer
Fully Trained = True
Dataset E:/Datasets/cor-splits/balanced_train
Learning Rate Epoch Schedule = 10
Learning Rate Gamma = 0.1
Optimizer = SGD
Batch Size = 32


Epoch 1/2
----------

train Loss: 0.6009 Acc: 0.6898
val Loss: 0.8255 Acc: 0.4899
Epoch duration: 0 m 28s
Learning Rate = 0.001



Epoch 2/2
----------

train Loss: 0.4133 Acc: 0.8668
val Loss: 0.8605 Acc: 0.5304
Epoch duration: 0 m 16s
Learning Rate = 0.001


Training complete in 0m 44s
Best val Acc: 0.530435

Confusio